In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV,train_test_split
from sklearn.model_selection import cross_validate

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [11]:
df_final = pd.read_table("data_frame_final_prognosis.tsv",sep=" ")

eln = [0]
comp =list(range(163,178)) 
age = [162]

all_gen = list(range(1,85))
tmp = df_final.iloc[:,all_gen][df_final.iloc[:,all_gen] >0].count()
gen = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

all_cyto = list(range(85,154))
tmp = df_final.iloc[:,all_cyto][df_final.iloc[:,all_cyto] >0].count()
cyto = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

clin =list(range(154,161))
demo = [161,162]
demo_without_age = [161]

eln_comp = eln + comp
eln_age = eln + age
eln_gen = eln + gen
eln_cyto = eln + cyto
eln_clin = eln + clin
eln_demo = eln + demo
eln_demo_without_age = eln + demo_without_age

# USEFUL FOR ELN COMPARISON
# with comp
eln_comp_age = eln_comp + age
eln_comp_gen = eln_comp + gen
eln_comp_cyto = eln_comp + cyto
eln_comp_clin = eln_comp + clin
eln_comp_demo = eln_comp + demo
eln_comp_demo_without_age = eln_comp + demo_without_age

eln_comp_age_gen = eln_comp_age + gen
eln_comp_age_cyto = eln_comp_age + cyto
eln_comp_age_clin = eln_comp_age + clin

eln_comp_gen_cyto = eln_comp_gen + cyto
eln_comp_gen_clin = eln_comp_gen + clin
eln_comp_gen_demo = eln_comp_gen + demo
eln_comp_gen_demo_without_age = eln_comp_gen + demo_without_age

eln_comp_cyto_clin = eln_comp_cyto + clin
eln_comp_cyto_demo = eln_comp_cyto + demo
eln_comp_cyto_demo_without_age = eln_comp_cyto + demo_without_age

eln_comp_clin_demo = eln_comp_clin + demo
eln_comp_clin_demo_without_age = eln_comp_clin + demo_without_age

eln_comp_age_gen_cyto = eln_comp_age_gen + cyto
eln_comp_age_gen_clin = eln_comp_age_gen + clin
eln_comp_age_gen_demo = eln_comp_age_gen + demo


eln_comp_gen_cyto_clin_demo = eln_comp_gen_cyto + clin + demo

# without comp

eln_age_gen = eln_age + gen
eln_age_cyto = eln_age + cyto
eln_age_clin = eln_age + clin

eln_gen_cyto = eln_gen + cyto
eln_gen_clin = eln_gen + clin
eln_gen_demo = eln_gen + demo
eln_gen_demo_without_age = eln_gen + demo_without_age

eln_cyto_clin = eln_cyto + clin
eln_cyto_demo = eln_cyto + demo
eln_cyto_demo_without_age = eln_cyto + demo_without_age

eln_clin_demo = eln_clin + demo
eln_clin_demo_without_age = eln_clin + demo_without_age

eln_age_gen_cyto = eln_age_gen + cyto
eln_age_gen_clin = eln_age_gen + clin
eln_age_gen_demo = eln_age_gen + demo


eln_gen_cyto_clin_demo = eln_gen_cyto + clin + demo

# USEFUL FOR COMP
comp_age = comp + age
comp_gen = comp + gen
comp_cyto = comp + cyto
comp_clin = comp + clin
comp_demo = comp + demo
comp_demo_without_age = comp + demo_without_age
comp_gen_cyto = comp_gen + cyto
comp_clin_demo = comp_clin + demo
comp_gen_cyto_clin_demo = comp_gen_cyto + clin + demo

#USEFUL FOR GEN
gen_age = gen + age
gen_cyto = gen + cyto
gen_clin = gen + clin
gen_demo = gen + demo
gen_demo_without_age = gen + demo_without_age
gen_clin_demo = gen_clin + demo
gen_cyto_clin_demo = gen_cyto + clin + demo

#USEFUL FOR CYTO 
cyto_age = cyto + age
cyto_clin = cyto + clin
cyto_demo = cyto + demo
gen_demo_without_age = gen + demo_without_age
cyto_clin_demo = cyto_clin + demo
cyto_gen_demo = gen_cyto + demo


clin_age  = clin + age


In [12]:
from contextlib import suppress

dict_features_type_final_comp = dict(zip(("comp","gen","cyto","clin","demo","eln_comp","eln_age","eln_gen","eln_cyto","eln_clin","eln_demo","eln_demo_without_age","eln_comp_age",
"eln_comp_gen","eln_comp_cyto","eln_comp_clin","eln_comp_demo","eln_comp_demo_without_age","eln_comp_age_gen",
"eln_comp_age_cyto","eln_comp_age_clin","eln_comp_gen_cyto","eln_comp_gen_clin","eln_comp_gen_demo",
"eln_comp_gen_demo_without_age","eln_comp_cyto_clin","eln_comp_cyto_demo","eln_comp_cyto_demo_without_age",
"eln_comp_clin_demo","eln_comp_clin_demo_without_age","eln_comp_age_gen_cyto","eln_comp_age_gen_clin",
"eln_comp_age_gen_demo","eln_comp_gen_cyto_clin_demo","eln_age_gen","eln_age_cyto","eln_age_clin","eln_gen_cyto","eln_gen_clin","eln_gen_demo","eln_gen_demo_without_age",
"eln_cyto_clin","eln_cyto_demo","eln_cyto_demo_without_age","eln_clin_demo","eln_clin_demo_without_age",
"eln_age_gen_cyto","eln_age_gen_clin","eln_age_gen_demo","eln_gen_cyto_clin_demo","comp_age","comp_gen","comp_cyto","comp_clin","comp_demo","comp_demo_without_age","comp_gen_cyto","comp_clin_demo",
 "comp_gen_cyto_clin_demo","gen_age","gen_cyto","gen_clin","gen_demo","gen_demo_without_age","gen_clin_demo","gen_cyto_clin_demo",
 "cyto_age","cyto_clin","cyto_demo","gen_demo_without_age","cyto_clin_demo","cyto_gen_demo","clin_age"),(comp,gen,cyto,clin,demo,eln_comp,eln_age,eln_gen,eln_cyto,eln_clin,eln_demo,eln_demo_without_age,eln_comp_age,
eln_comp_gen,eln_comp_cyto,eln_comp_clin,eln_comp_demo,eln_comp_demo_without_age,eln_comp_age_gen,
eln_comp_age_cyto,eln_comp_age_clin,eln_comp_gen_cyto,eln_comp_gen_clin,eln_comp_gen_demo,
eln_comp_gen_demo_without_age,eln_comp_cyto_clin,eln_comp_cyto_demo,eln_comp_cyto_demo_without_age,
eln_comp_clin_demo,eln_comp_clin_demo_without_age,eln_comp_age_gen_cyto,eln_comp_age_gen_clin,
eln_comp_age_gen_demo,eln_comp_gen_cyto_clin_demo,eln_age_gen,eln_age_cyto,eln_age_clin,eln_gen_cyto,eln_gen_clin,eln_gen_demo,eln_gen_demo_without_age,
eln_cyto_clin,eln_cyto_demo,eln_cyto_demo_without_age,eln_clin_demo,eln_clin_demo_without_age,
eln_age_gen_cyto,eln_age_gen_clin,eln_age_gen_demo,eln_gen_cyto_clin_demo,comp_age,comp_gen,comp_cyto,comp_clin,comp_demo,comp_demo_without_age,comp_gen_cyto,comp_clin_demo,
 comp_gen_cyto_clin_demo,gen_age,gen_cyto,gen_clin,gen_demo,gen_demo_without_age,gen_clin_demo,gen_cyto_clin_demo,
 cyto_age,cyto_clin,cyto_demo,gen_demo_without_age,cyto_clin_demo,cyto_gen_demo,clin_age)))

In [16]:
df = pd.DataFrame(columns=dict_features_type_final_comp.keys())
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5.5,-5,-4.5,-2.5,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)
for key,item in dict_features_type_final_comp.items():
    
    print(key)
    
    ##############    FEATURES REDUCED DF   ##################                  
    with suppress(Exception):              
        tmp = pd.read_table("Bootstrap_coef/" + key + "_bootstrap_glm_0.7.tsv")
        tmp1 = tmp[tmp.coef>0]
        tmp2 = tmp[tmp.coef<0]
        cols = tmp1[tmp1.coef>tmp1.coef.quantile(0.9)].feature.tolist()+tmp2[tmp2.coef<tmp2.coef.quantile(0.15)].feature.tolist()+tmp[tmp.n>tmp.n.quantile(0.85)].feature.tolist()
        x_reduced = df_final[cols]
        response = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
        ci=[]
        print(response)
        for i in range(10):
            X_train, X_test, y_train, y_test = train_test_split(x_reduced, response, test_size=0.2, random_state=i)
            gcv = gcv.fit(X_train,y_train)
            print(gcv.best_params_)
            ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
            print(ci)
        df[key] = ci
        
##############    BOOTSTRAPPING DF   ##################

    df_svm = pd.DataFrame()
    with suppress(Exception):
        for i in range(100):##100
            data = df_final.sample(frac=0.8,replace=True,random_state=i)
            x = data.iloc[:,item]
            y = np.array(list(zip(data.os_status, data.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
            est = FastSurvivalSVM(alpha=gcv.best_params_["alpha"], fit_intercept=False, max_iter=1000, optimizer=gcv.best_params_["optimizer"],
                    random_state=17, rank_ratio=1.0, timeit=False, tol=1e-06,
                    verbose=False).fit(x,y)
            df_svm = df_svm.append(pd.DataFrame({'feature': x.columns, 'coef': est.coef_}),ignore_index=True)
        df_svm = df_svm.groupby(['feature']).sum()
        df_svm["feature"] = df_svm.index
        df_svm["n"] = 100    
        del df_svm.index.name

        df_svm.to_csv("Bootstrap_coef/" + key + "_bootstrap_SVM.csv")
        #df_svm.to_csv( key + "_bootstrap_SVM.csv")

df.to_csv("Bootstrap_coef/SVM_reduced.csv")

comp
[( True, 0.19164956) (False, 4.7063655 ) ( True, 0.37782341) ...
 ( True, 1.44010951) ( True, 0.07392197) ( True, 0.41341547)]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078, 0.5457180907215825]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078, 0.5457180907215825, 0.5770456960680127]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078, 0.5457180907215825, 0.5770456960680127, 0.5696183590858259]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078, 0.5457180907215825, 0.5770456960680127, 0.5696183590858259, 0.5608963036706381]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078, 0.5457180907215825, 0.5770456960680127, 0.5696183590858259, 0.5608963036706381, 0.5537295999480873]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5390591957894078, 0.5457180907215825, 0.5770456960680127, 0.5696183590858259, 0.5608963036706381, 0.5537295999480873, 0.5498582382885495]
{'alpha': 1e-0

/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.


gen
[( True, 0.19164956) (False, 4.7063655 ) ( True, 0.37782341) ...
 ( True, 1.44010951) ( True, 0.07392197) ( True, 0.41341547)]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5526010745782359]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.5526010745782359, 0.5647767690383773]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5526010745782359, 0.5647767690383773, 0.6037772707382634]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5526010745782359, 0.5647767690383773, 0.6037772707382634, 0.5835026540030557]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.5526010745782359, 0.5647767690383773, 0.6037772707382634, 0.5835026540030557, 0.5759658306303992]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.5526010745782359, 0.5647767690383773, 0.6037772707382634, 0.5835026540030557, 0.5759658306303992, 0.5631549917264203]
{'alpha': 3.162277660168379e-06, 'optimizer': 'avltree'}
[0.5526010745782359, 0.5647767690383773, 0.6037772707382634, 0.5835026540030557, 0.5759658306303992, 0.5631549917264203, 0.5

/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.


demo
[( True, 0.19164956) (False, 4.7063655 ) ( True, 0.37782341) ...
 ( True, 1.44010951) ( True, 0.07392197) ( True, 0.41341547)]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.690158053854227]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.690158053854227, 0.6814431650236304]


...

/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.


cyto_gen_demo
[( True, 0.19164956) (False, 4.7063655 ) ( True, 0.37782341) ...
 ( True, 1.44010951) ( True, 0.07392197) ( True, 0.41341547)]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7120960541361864]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7120960541361864, 0.6971724619557274]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7120960541361864, 0.6971724619557274, 0.6861755329124211]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7120960541361864, 0.6971724619557274, 0.6861755329124211, 0.7296697065633417]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7120960541361864, 0.6971724619557274, 0.6861755329124211, 0.7296697065633417, 0.7102989819840072]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.7120960541361864, 0.6971724619557274, 0.6861755329124211, 0.7296697065633417, 0.7102989819840072, 0.6942182278316732]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7120960541361864, 0.6971724619557274, 0.6861755329124211, 0.7296697065633417, 0.7102989819840072, 0.6942182278316732, 0.7049181

/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.


clin_age
[( True, 0.19164956) (False, 4.7063655 ) ( True, 0.37782341) ...
 ( True, 1.44010951) ( True, 0.07392197) ( True, 0.41341547)]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.5860210842901674]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.5860210842901674, 0.6137917593750507]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.5860210842901674, 0.6137917593750507, 0.5987997749578046]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.5860210842901674, 0.6137917593750507, 0.5987997749578046, 0.6031674778308054]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.5860210842901674, 0.6137917593750507, 0.5987997749578046, 0.6031674778308054, 0.5630158322361026]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.5860210842901674, 0.6137917593750507, 0.5987997749578046, 0.6031674778308054, 0.5630158322361026, 0.6083108919243373]
{'alpha': 0.0031622776601683794, 'optimizer': 'avltree'}
[0.5860210842901674, 0.6137917593750507, 0.5987997749578046, 0.60316747

/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/Users/taziy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
/User

In [17]:
df.to_csv("Bootstrap_coef/SVM_reduced.csv")